In [2]:
#importing libraries
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from collections import Counter
import seaborn as sns
from pymongo import MongoClient

In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = MongoClient(conn)

In [4]:
beer_df = pd.read_csv('beer_reviews (1).csv')
beer_df.head(-1)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586608,14359,The Defiant Brewing Company,1162685856,3.5,4.0,4.0,treehugger02010,Pumpkin Ale,3.5,3.0,The Horseman's Ale,5.2,33061
1586609,14359,The Defiant Brewing Company,1162684892,5.0,4.0,3.5,maddogruss,Pumpkin Ale,4.0,4.0,The Horseman's Ale,5.2,33061
1586610,14359,The Defiant Brewing Company,1161048566,4.0,5.0,2.5,yelterdow,Pumpkin Ale,2.0,4.0,The Horseman's Ale,5.2,33061
1586611,14359,The Defiant Brewing Company,1160702513,4.5,3.5,3.0,TongoRad,Pumpkin Ale,3.5,4.0,The Horseman's Ale,5.2,33061


In [5]:
#shape of beer data
print("Number of reviews: ", beer_df.shape[0])
print("Number of features: ", beer_df.shape[1])
print("Total number of breweries; ", len(beer_df['brewery_name'].value_counts()))
print("Total number of beer style; ", len(beer_df['beer_style'].value_counts()))
print("Total number of beer names; ", len(beer_df['beer_name'].value_counts()))
print("Total number of brewery names; ", len(beer_df['brewery_name'].value_counts()))
print("Total number of brewery ID; ", len(beer_df['brewery_id'].value_counts()))

Number of reviews:  1586614
Number of features:  13
Total number of breweries;  5742
Total number of beer style;  104
Total number of beer names;  56857
Total number of brewery names;  5742
Total number of brewery ID;  5840


In [6]:
# missing data by rows
beer_row_clean = beer_df.dropna(subset=['brewery_id','brewery_name', 'beer_name', 'review_taste', 'review_overall', 'review_palate', 'review_aroma','beer_style', 'review_appearance'], how='any')
beer_row_clean.head(800000)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
...,...,...,...,...,...,...,...,...,...,...,...,...,...
800004,18149,The Lost Abbey,1256157007,4.5,4.5,4.0,Viggo,American Wild Ale,4.5,4.5,Duck Duck Gooze,7.0,51116
800005,18149,The Lost Abbey,1256057246,4.0,4.0,4.0,grub,American Wild Ale,4.0,4.5,Duck Duck Gooze,7.0,51116
800006,18149,The Lost Abbey,1255289201,4.5,4.0,4.0,pwoods,American Wild Ale,4.5,4.5,Duck Duck Gooze,7.0,51116
800007,18149,The Lost Abbey,1254368506,5.0,4.5,4.5,Georgiabeer,American Wild Ale,4.5,4.5,Duck Duck Gooze,7.0,51116


In [12]:
# Main Data Tables

# brewery_df will have all the brewery information, specifically the id, and the brewery name.
brewery_df = beer_df[['brewery_id','brewery_name']].drop_duplicates(keep='first')
brewery_df1 = brewery_df.head(100000)
# convert brewery_df to the list of doctionary inorder to import them to MongoDB
table_1 = []

# for brewery in beer_df:
#     brewery_id = brewery_id.find('div', class_='content_title')
#     brewery_name = brewery_id.find('div', class_='article_teaser_body')
# result = {"Brewery_ID": brewery_df1['brewery_id'].to_list(),
#           "Brewery_Name": brewery_df1['brewery_name'].to_list()}
#     table_1.append(result)
# result
# review_df will have all the various reviews for each of the beer id's or beer_beerid
# review_df = brewery_df1[['beer_beerid', 'brewery_id','review_overall','review_aroma','review_appearance','review_palate','review_taste']].groupby('beer_beerid').mean().round(2)
# review_df
# brewery_df1


,brewery_id,brewery_name
0,10325,Vecchio Birraio
4,1075,Caldera Brewing Company
10,163,Amstel Brouwerij B. V.
393,1454,Broad Ripple Brew Pub
403,850,Moon River Brewing Company
...,...,...
1586294,21727,Gattopardo Cervejaria
1586297,6785,Brauerei Lasser GmbH
1586302,12566,Wissey Valley Brewery
1586303,7337,Outback Brewery Pty Ltd


In [32]:
# import data to MongoDB
from pymongo import MongoClient
import json 
records= json.loads(brewery_df.to_json(orient='records'))

client = MongoClient()
client
client = MongoClient(host="localhost", port=27017)
db = client.BeerAdvocate_DB
collection = db.brewerydata

results = collection.insert_many(records)
results

In [33]:
# review_df will have all the various reviews for each of the beer id's or beer_beerid
review_df = beer_df[['beer_beerid', 'brewery_id','review_overall','review_aroma','review_appearance','review_palate','review_taste']].groupby('beer_beerid').mean().round(2)
review_df.reset_index()

,beer_beerid,brewery_id,review_overall,review_aroma,review_appearance,review_palate,review_taste
0,3,2.0,4.17,4.00,3.83,4.17,4.17
1,4,2.0,3.70,3.80,3.95,3.70,3.45
2,5,3.0,3.55,3.21,3.49,3.32,3.34
3,6,3.0,3.71,3.52,3.84,3.51,3.65
4,7,3.0,3.27,3.18,3.25,3.11,3.10
...,...,...,...,...,...,...,...
66050,77313,185.0,3.00,3.00,4.00,3.00,3.00
66051,77314,18604.0,3.50,3.50,4.00,3.00,4.00
66052,77315,27839.0,3.50,4.50,3.00,3.00,3.50
66053,77316,20330.0,3.50,3.50,3.50,3.50,3.50


In [34]:
# import data to MongoDB
from pymongo import MongoClient
import json 
records= json.loads(review_df.to_json(orient='records'))
# print (records)

client = MongoClient()
client
client = MongoClient(host="localhost", port=27017)
db = client.BeerAdvocate_DB
collection = db.reviewdata

results = collection.insert_many(records)
results

In [35]:
# beerid_df will have all the beer information such as name, style, and alcohol content for each beer_id
beerid_df = beer_df[['beer_beerid','beer_name','beer_style','beer_abv']].drop_duplicates('beer_beerid', keep='first').set_index('beer_beerid').sort_index()

In [36]:
# import data to MongoDB
from pymongo import MongoClient
import json 
records= json.loads(review_df.to_json(orient='records'))
# print (records)

client = MongoClient()
client
client = MongoClient(host="localhost", port=27017)
db = client.BeerAdvocate_DB
collection = db.beerdata

results = collection.insert_many(records)
results

In [30]:
# Which brewry produces the strongest beers by ABV%? (the mean of ABV for each brewry has beed considered due to the lenght of dataset)
data = beer_df.groupby('brewery_name').beer_abv.mean()
strong = pd.DataFrame(data.reset_index())

strong.columns = ['brewery_name', 'beer_abv_mean']

strongest = strong[strong.beer_abv_mean == strong.beer_abv_mean.max()]
strongest

,brewery_name,beer_abv_mean
4667,Schorschbräu,19.228824


In [32]:
# If asked pick the Top 3 beer consider following factors:
# 1)review_overall, 
# 2) review_aroma, 
# 3) review_appearance 
# 4) review_palate, 
# 5) review_taste
# 6) beer_abv

reviews = beer_df.groupby('beer_beerid').agg({'review_overall': np.mean,
                                'review_aroma': np.mean,
                                'review_appearance': np.mean,
                                'review_palate': np.mean,     
                                'review_taste': np.mean,
                                'beer_abv': np.mean})

picks = pd.DataFrame(reviews.reset_index())

finaltop = picks.sort_values(['review_overall', 'review_aroma','review_appearance','review_palate','review_taste','beer_abv'], ascending=[False,False, False, False, False, False])
finaltop[:5]

,beer_beerid,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv
46212,55451,5.0,5.0,5.0,5.0,5.0,13.0
64816,75956,5.0,5.0,5.0,5.0,5.0,11.0
40090,48325,5.0,5.0,5.0,5.0,5.0,10.5
1477,1734,5.0,5.0,5.0,5.0,5.0,10.0
1608,1890,5.0,5.0,5.0,5.0,5.0,10.0
